# Introduction and setup

In this notebook, I have code to load in the stellar profiles in 3 dimensions which 
were previously calculated on the command line so that we can plot quantities such as 
the 1D velocity dispersion and evolution with time of the mass.

In [ ]:
using Revise
using LilGuys
using CairoMakie, Arya

In [ ]:
modelsdir = ENV["DWARFS_ROOT"] * "/analysis"

In [ ]:
import TOML
obs_props = TOML.parsefile(joinpath(ENV["DWARFS_ROOT"], "observations/sculptor/observed_properties.toml"))

In [ ]:
function load_profile(modelname, starsname)
    orbit = load_profiles(modelname * "/stars/" * starsname)

    return orbit
end

In [ ]:
function load_isolation(modelname, starsname)
    iso = load_profiles(modelname * "/../stars/" * starsname)
    return iso
end

In [ ]:
function load_profiles(filename, starsname; extension="stellar_profiles_3d.hdf5")
    path = joinpath(modelsdir, filename, "stars", starsname, extension)
    
    profs = LilGuys.read_structs_from_hdf5(path, LilGuys.StellarProfile3D)
    idxs = parse.(Int, first.(profs))
    profs = last.(profs)

    s = sortperm(idxs)
    return [idxs[i] => profs[i] for i in s ]
end

In [ ]:
function load_prof_expected(filename, parampath="profile.toml")
    path = joinpath(modelsdir, filename, parampath)
        
    if isfile(parampath)
        expected = LilGuys.load_profile(parampath)
    else
        error("file not found $path")
    end

    return expected
end

In [ ]:
function compare_profiles(profiles, expected=nothing; 
        sequential=false, legend=true, limits=(-1.5, 0.8, -15, 3), 
        kwargs...
    )
    
    fig = Figure()

	ax = Axis(fig[1,1], xlabel=L"\log\, r / \textrm{kpc}", ylabel =  L"\log\, \rho_\star\; [10^{10} M_\odot / \textrm{kpc}^3]", 
		limits=limits; kwargs...
		)

    plot_kwargs = Dict{Symbol, Any}()

    if sequential
        plot_kwargs[:colorrange] = (1, length(profiles))
    end
    
    for i in eachindex(profiles)
        label, profs = profiles[i]
        prof = profs[end].second
        x = prof.log_r
        y = log10.(prof.rho)
        if sequential
            plot_kwargs[:color] = i
        end

    	lines!(x, y, label=string(label); plot_kwargs...)
        arrows!([log10.(prof.r_break)],[-10], [0], [3], color=COLORS[i])
    end

    if expected !== nothing
    	log_r_pred = LinRange(-2, 2, 1000)
    	ρ_s_pred = calc_ρ.(expected, 10 .^ log_r_pred)

    	lines!(log_r_pred, log10.(ρ_s_pred), label="expected", color="black", linestyle=:dot)
    end

    if legend
    	axislegend(ax, position=:lb)
    end 
    
	fig
end

In [ ]:
function plot_v_t!(profiles; x_shift = 0, kwargs...)
	sigmas = [p.second.sigma_vx for p in profiles]

    t = [p.second.time for p in profiles] 
	scatterlines!(t * T2GYR .+ x_shift, sigmas * V2KMS; kwargs...)
end

In [ ]:
function plot_v_t(profiles; x_shift=0, kwargs...)
	fig, ax = FigAxis(
		xlabel = "time / Gyr",
		ylabel = L"\sigma_v / \textrm{km s^{-1}}";
        kwargs...
	)

    plot_v_t!(profiles, x_shift=x_shift)

    fig
end
    

In [ ]:
function compare_v_t(profilesi, profs_iso=nothing; x_shifts=zeros(length(profilesi)), legend_position=:lb)
	fig, ax = FigAxis(
		xlabel = "time / Gyr",
		ylabel = L"\sigma_v / \textrm{km s^{-1}}",
	)


    for i in eachindex(profilesi)
        label, profiles = profilesi[i]

        kwargs = Dict{Symbol, Any}()
        kwargs[:label] = label
        kwargs[:color] = COLORS[i]
  
        plot_v_t!(profiles; x_shift=x_shifts[i], kwargs...)
    end

    if profs_iso !== nothing
        for i in eachindex(profs_iso)
            label, profiles = profs_iso[i]
    
      
            sigmas = [p.second.sigma_vx for p in profiles]
    
            t = [p.second.time for p in profiles] 
            t = t .- t[end]
            scatterlines!(t * T2GYR, sigmas * V2KMS, color=COLORS[i])
            
        end
    end
    
    hlines!(obs_props["sigma_v"], color=:grey)
    hspan!(obs_props["sigma_v"] - obs_props["sigma_v_err"], 
        obs_props["sigma_v"] + obs_props["sigma_v_err"],
        color = (:grey, 0.1), label="observed")

    LilGuys.hide_grid!(ax)
    if legend_position == :outside
        Legend(fig[1,2], ax)
    else
        
        axislegend(position=legend_position)
    end
    
	fig
end

In [ ]:
function compare_profiles_t(profiles, expected=nothing; 
        sequential=false, legend=true, limits=(-2.5, 1.0, -6, 3), 
        skip=1,
        kwargs...
    )
    
    fig = Figure()
	ax = Axis(fig[1, 1]; limits=limits, kwargs...)

    plot_kwargs = Dict{Symbol, Any}()

    if sequential
        if profiles[1].second.time !== NaN
            has_time = true
            plot_kwargs[:colorrange] = (profiles[1].second.time * T2GYR, profiles[end].second.time * T2GYR)
        else
            
            plot_kwargs[:colorrange] = (1, length(profiles))
            has_time = false
        end
        
    end

    local p
    
    for i in eachindex(profiles)[1:skip:end]
        label, prof = profiles[i]
        x = prof.log_r
        y = log10.(prof.rho)
        if sequential
            if has_time
                plot_kwargs[:color] = prof.time * T2GYR
            else
                plot_kwargs[:color] = i
            end
            
        end

    	p = lines!(x, y, label=string(label); plot_kwargs...)
    end

    if expected !== nothing
    	log_r_pred = LinRange(-2, 2, 1000)
    	ρ_s_pred = lguys.calc_Σ.(expected, 10 .^ log_r_pred)

    	lines!(log_r_pred, log10.(ρ_s_pred), label="expected", color="black", linestyle=:dot)
    end

    if legend
    	axislegend(ax, position=:lb)
    end 
    if sequential
        if has_time 
            label = "time / Gyr"
        else
            label = "snapshot"
        end
        
        Colorbar(fig[1,2], colorrange=plot_kwargs[:colorrange], label=label)
    end
    
	fig
end

# 1e7

In [ ]:
profs = [
    "exp2d" => load_profile("sculptor/1e7_V31_r3.2/orbit_mean", "exp2d_rs0.13"),
 "plummer" => load_profile("sculptor/1e7_V31_r3.2/orbit_mean", "plummer_rs0.20"),

];

In [ ]:
profs_iso = [
    "exp2d" => load_isolation("sculptor/1e7_V31_r3.2/orbit_mean", "exp2d_rs0.13"),
    "plummer" => load_isolation("sculptor/1e7_V31_r3.2/orbit_mean", "plummer_rs0.20"),
];

In [ ]:
compare_v_t(profs, profs_iso)

In [ ]:
compare_profiles(profs)

In [ ]:
compare_profiles(profs_iso)

### Comparison between orbits

In [ ]:
profs = [
    "mean" => load_profile("sculptor/1e7_V31_r3.2/orbit_mean", "exp2d_rs0.13"),
    "smallperi" => load_profile("sculptor/1e7_V31_r3.2/orbit_smallperi", "exp2d_rs0.08"),
    ];

In [ ]:
profs_iso = [
    "mean" => load_isolation("sculptor/1e7_V31_r3.2/orbit_mean", "exp2d_rs0.13"),
    "smallperi" => load_isolation("sculptor/1e7_V31_r3.2/orbit_smallperi", "exp2d_rs0.08"),
];

In [ ]:
compare_v_t(profs, profs_iso)

In [ ]:
profs = [
    "mean" => load_profile("sculptor/1e7_V31_r3.2/orbit_mean", "plummer_rs0.20"),
    "smallperi" => load_profile("sculptor/1e7_V31_r3.2/orbit_smallperi", "plummer_rs0.18"),
    ];

In [ ]:
profs_iso = [
    "mean" => load_isolation("sculptor/1e7_V31_r3.2/orbit_mean", "plummer_rs0.20"),
    "smallperi" => load_isolation("sculptor/1e7_V31_r3.2/orbit_smallperi", "plummer_rs0.18"),
];

In [ ]:
compare_v_t(profs, profs_iso)

# 1e6

In [ ]:
profs = [
    "mean" => load_profile("sculptor/1e6_V31_r3.2/orbit_mean", "exp2d_rs0.13"),
    "smallperi" => load_profile("sculptor/1e6_V31_r3.2/orbit_smallperi", "exp2d_rs0.13"),
    "V+21" => load_profile("sculptor/1e6_V31_r3.2/vasiliev+21_mean", "exp2d_rs0.13"),
    "V+21 small" => load_profile("sculptor/1e6_V31_r3.2/vasiliev+21_smallperi", "exp2d_rs0.13"),
    "V+21b" => load_profile("sculptor/1e6_V31_r4.2/vasiliev+21_smallperi", "exp2d_rs0.13"),
    ];

In [ ]:
profs = load_profiles("sculptor/1e6_V31_r3.2/orbit_mean", "exp2d_rs0.13");

In [ ]:
compare_profiles_t(profs, sequential=true, limits=(-2, 1.5, -20, 3))

In [ ]:
profs = load_profiles("sculptor/1e6_V31_r3.2/vasiliev+21_smallperi/", "exp2d_rs0.13");

In [ ]:
compare_profiles_t(profs, sequential=true, limits=(-2, 1.5, -20, 3))

In [ ]:
profs_iso = [
    "mean" => load_isolation("sculptor/1e6_V31_r3.2/orbit_mean", "exp2d_rs0.13"),
    "smallperi" => load_isolation("sculptor/1e6_V31_r3.2/orbit_smallperi", "exp2d_rs0.13"),
    "V+21" => load_isolation("sculptor/1e6_V31_r3.2/vasiliev+21_mean", "exp2d_rs0.13"),
    "V+21 small" => load_isolation("sculptor/1e6_V31_r3.2/vasiliev+21_smallperi", "exp2d_rs0.13"),
    "V+21b" => load_isolation("sculptor/1e6_V31_r4.2/vasiliev+21_smallperi", "exp2d_rs0.13"),
];

In [ ]:
compare_v_t(profs, profs_iso, x_shifts=[0, 0, 4.977,4.977,4.977, ], legend_position=:rt)

In [ ]:
profs = [
    "V+21" => load_profile("sculptor/1e6_V31_r3.2/vasiliev+21_mean", "exp2d_rs0.13"),
    "V+21 small" => load_profile("sculptor/1e6_V31_r3.2/vasiliev+21_smallperi", "exp2d_rs0.13"),
    "V+21 heavylmc" => load_profile("sculptor/1e6_V31_r3.2/vasiliev+21_heavylmc", "exp2d_rs0.13"),
    "V+21 heavylmc 2peri" => load_profile("sculptor/1e6_V31_r3.2/vasiliev+21_heavylmc_smallperilmc", "exp2d_rs0.13"),
    ];

In [ ]:
profs_iso = [
    "V+21" => load_isolation("sculptor/1e6_V31_r3.2/vasiliev+21_mean", "exp2d_rs0.13"),
    "V+21 small" => load_isolation("sculptor/1e6_V31_r3.2/vasiliev+21_smallperi", "exp2d_rs0.13"),
    "V+21 heavylmc" => load_isolation("sculptor/1e6_V31_r3.2/vasiliev+21_heavylmc", "exp2d_rs0.13"),
    "V+21 heavylmc 2peri" => load_isolation("sculptor/1e6_V31_r3.2/vasiliev+21_heavylmc_smallperilmc", "exp2d_rs0.13"),
];

In [ ]:
compare_v_t(profs, profs_iso, x_shifts=[4.977,4.977,4.977, 4.977], legend_position=:lb)

In [ ]:
profs = [
    "V+21" => load_profile("sculptor/1e6_V31_r3.2/vasiliev+21_mean", "exp2d_rs0.13"),
    "L3M11" => load_profile("sculptor/1e6_V31_r3.2/vasiliev24_L3M11_smallperi", "exp2d_rs0.13"),
    ];

In [ ]:
compare_profiles(profs[end].second)

In [ ]:
profs_iso = [
    "V+21" => load_isolation("sculptor/1e6_V31_r3.2/vasiliev+21_mean", "exp2d_rs0.13"),
    "L3M11" => load_isolation("sculptor/1e6_V31_r3.2/vasiliev24_L3M11_smallperi", "exp2d_rs0.13"),
];

In [ ]:
compare_v_t(profs, profs_iso, x_shifts=[4.977,10], legend_position=:lb)

In [ ]:
compare_v_t(profs, legend_position=:lb)

In [ ]:
profs = load_profiles("sculptor/1e6_V31_r3.2/vasiliev24_L2M11_extremeperi", "exp2d_rs0.13");

In [ ]:
compare_profiles_t(profs, sequential=true, limits=(-2, 1, -15, 3))


In [ ]:
plot_v_t(profs)

In [ ]:
prof_i = profs[1].second

In [ ]:
prof_i.